Medoids

In [3]:
%load_ext google.cloud.bigquery
%matplotlib inline
from google.cloud import bigquery
client = bigquery.Client()
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import cluster
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import random
from sklearn.cluster import KMeans
import matplotlib.cm as cm

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [ ]:
sql = '''
SELECT *  FROM `eth-tokens.alldata.clean_avg`
'''

df = client.query(sql).to_dataframe()

In [ ]:
#remove address (string) from analysis
data = df.iloc[:,1:]

In [ ]:
num_features = data.shape[1]
num_clusters = 12

In [ ]:
scale = StandardScaler()
pca =PCA(n_components=num_features)

#build pipeline
pipe = Pipeline([('scale', scale ),
                 ('PCA', pca)])

results = pipe.fit_transform(data)





In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
import kmedoids


# distance matrix
D = pairwise_distances(results, metric='euclidean')

# split into 4 clusters
M, C = kmedoids.kMedoids(D, num_clusters)



In [ ]:
#plot k medoids clusters against two principal components
for lbl in C:
    a = results[C[lbl]]
    plt.scatter(a[:,0],a[:,1], label = lbl, alpha=.2)
plt.legend()
# plt.xlim(0,20)
# plt.ylim(-10,10)

In [ ]:
#get medoids
m = results[M]
#transform back to real numbers
medoids = scale.inverse_transform(pca.inverse_transform(m))
pd.DataFrame(medoids, columns = data.columns)